In [3]:
#要搜尋的網站的driver
import requests
#將抓到的json檔案寫入json中
import json
#新增資料夾
import os
#存圖
import urllib.request 
#F12
from bs4 import BeautifulSoup
#sleep
import time
import re
Host = ("https://www.dcard.tw/_api/posts/")
FURL = ("https://www.dcard.tw/f/sex")
comments = ("/comments")
popularT = ("/comments?popular=true")
imgHost = 'https://i.imgur.com/'
headers = {"cookie":"__cfduid=d4ba3da2ed21be62be47991532506702b1493536192; dcard-web=eyJjc3JmU2VjcmV0IjoiSS1jM29RX28yY2VsbzhqMUM2UEFYUmkwIiwidG9rZW4iOiJXNFBrMjRyd1NWT1RNUjY0dkRZUjBBPT0ifQ==; dcard-web.sig=6bzkzcF0qJ6OZWxYAeLAunj6oA4; appier_utmz=%7B%22csr%22%3A%22google%22%2C%22timestamp%22%3A1493536194%7D; _atrk_sync_cookie=true; _atrk_ssid=gBdOKGCeHGRnZL5rCElE8K; _atrk_sessidx=3; _atrk_siteuid=l6CVbjamd81L-bof; __asc=e34e3a1d15bbdb1bcb32cb74699; __auc=e34e3a1d15bbdb1bcb32cb74699; _ga=GA1.2.304721922.1493536194; __gads=ID=af285be025863ded:T=1493536199:S=ALNI_MZTwWkZTnYhWA4ajl_E_74NmHi-kQ; _gat=1"}
res = requests.get(FURL, headers=headers)
soup = BeautifulSoup(res.text, "lxml")
hrefs = soup.select('a.PostEntry_entry_2rsgm')
#抓最新的前28篇
for i in range(3,len(hrefs)):
    #本文
    href = hrefs[i]['href'].split('/f/sex/p/')[1].split('-')[0]
    url = ("{}{}?".format(Host, href))
    #推文
    com = ("{}{}{}".format(Host, href, comments))
    #讚最多
    pop = ("{}{}{}".format(Host, href, popularT))

    conres = requests.get(url)
    time.sleep(3)
    comres = requests.get(com)
    time.sleep(3)
    popres = requests.get(pop)
    time.sleep(3)
    
    #將json檔存入json物件中
    conload = json.loads(conres.text)
    comload = json.loads(comres.text)
    popload = json.loads(popres.text)
    
    #Dcard可以直接抓到json格式的網址
    #所以直接選取需要的key，回傳值式value
    #可以print(load)出來看是什麼樣子
    content = str(conload['content']).strip('\n')
    title = str(conload['title']).replace('/','')   
    path = "/home/ubuntu/Downloads/testDC/{}".format(title)
  
    #建立資料夾，若資料夾不存在，則mkdir
    if not os.path.isdir(path):
        os.mkdir(path)
        #開啟一個檔案("路徑+檔案類型"，"是讀還是寫"，"檔案編碼")    
    con = open('/home/ubuntu/Downloads/testDC/{}/{}.txt'.format(title,title), 'w', encoding = 'UTF-8')        
    #將資料寫入開啟的檔案
    con.write(content)  
    
    #建立圖檔的URL和名稱的list       
    img_url = []
    img_name = []
    #因為一個網站有很多圖
    for no in range(0,len(conload['media'])):
        #圖的URL
        img_url.append(conload['media'][no]['url'])
        #圖的名稱
        img_name.append(conload['media'][no]['url'].split('com/')[1].split('.jpg')[0])
        #將圖寫入硬體("要寫入的URL"，"要寫入的路徑")
        urllib.request.urlretrieve(img_url[no],'/home/ubuntu/Downloads/testDC/{}/{}.jpg'.format(title,img_name[no]))
    
    comcon = []
    for i in range(0,len(comload)):
        try:
            comcon.append(comload[i]['content'])
            con.write(str(comcon))
            booimg = comload[i]['content'].index(imgHost)
            if booimg>=0:
                url = re.findall('https://i.imgur.com/([a-zA-z0-9]*.jpg)',comload[i]['content'])
                #將圖寫入硬體("要寫入的URL"，"要寫入的路徑")
                urllib.request.urlretrieve('{}{}'.format(imgHost, url[0]),'/home/ubuntu/Downloads/testDC/{}/{}.jpg'.format(title,url[0]))
                
        except:
            continue
            
    popcon = []
    for i in range(0,len(popload)):
        try:
            popcon = popload[i]['content']
            con.write(str(popcon))
            booimg = popload[i]['content'].index(imgHost)
            if booimg>=0:
                url = re.findall('https://i.imgur.com/([a-zA-z0-9]*.jpg)',popload[i]['content'])
                #將圖寫入硬體("要寫入的URL"，"要寫入的路徑")
                urllib.request.urlretrieve('{}{}'.format(imgHost, url[0]),'/home/ubuntu/Downloads/testDC/{}/{}.jpg'.format(title,url[0]))
        except:
            continue
            
    #關閉資源
    con.close()

KeyboardInterrupt: 